<a href="https://colab.research.google.com/github/harirm85/Hari_ML_workshop/blob/main/Home_Credit_App_CV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Cross Vlidation of XG Boost

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.float_format = '{:,.2f}'.format

In [2]:
# Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Copy the Jason file and set security
!mkdir -p ~/.kaggle
!cp '/content/drive/MyDrive/Kaggle/kaggle.json' ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

#Download data
!kaggle competitions download -c home-credit-default-risk

import zipfile

# Specify the path to the downloaded zip file
zip_path = "/content/home-credit-default-risk.zip"

# Specify the directory to extract the files to
extract_path = "/content/home-credit-default-risk"

# Extract the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

Mounted at /content/drive
 99% 683M/688M [00:03<00:00, 236MB/s]
100% 688M/688M [00:03<00:00, 188MB/s]


In [3]:
#Take app train data and load to DF
app_data = pd.read_csv('/content/home-credit-default-risk/application_train.csv')

print(app_data.shape)
print(app_data.columns)

(307511, 122)
Index(['SK_ID_CURR', 'TARGET', 'NAME_CONTRACT_TYPE', 'CODE_GENDER',
       'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL',
       'AMT_CREDIT', 'AMT_ANNUITY',
       ...
       'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20',
       'FLAG_DOCUMENT_21', 'AMT_REQ_CREDIT_BUREAU_HOUR',
       'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK',
       'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT',
       'AMT_REQ_CREDIT_BUREAU_YEAR'],
      dtype='object', length=122)


In [12]:
#Set Features and label fields
label_col = 'TARGET'
feature_columns = app_data.columns
feature_columns = feature_columns.drop (['SK_ID_CURR', 'TARGET']) 

#Check duplicates
duplicates = app_data.duplicated(feature_columns).sum(axis=0)
print(duplicates) #no dups for all fields

#Fix data types
dtypes= {'NAME_CONTRACT_TYPE': 'category' ,'CODE_GENDER': 'category','FLAG_OWN_CAR': 'category','FLAG_OWN_REALTY': 'category'
  , 'CNT_CHILDREN': 'category' , 'TARGET': 'category' , 'SK_ID_CURR': 'category'
  ,'NAME_TYPE_SUITE': 'category','NAME_INCOME_TYPE': 'category','NAME_EDUCATION_TYPE': 'category','NAME_FAMILY_STATUS': 'category'
  ,'NAME_HOUSING_TYPE': 'category','FLAG_MOBIL': 'category','FLAG_EMP_PHONE': 'category','FLAG_WORK_PHONE': 'category'
  ,'FLAG_CONT_MOBILE': 'category','FLAG_PHONE': 'category','FLAG_EMAIL': 'category','OCCUPATION_TYPE': 'category'
  ,'CNT_FAM_MEMBERS': 'category','REGION_RATING_CLIENT': 'category','REGION_RATING_CLIENT_W_CITY': 'category'  
  ,'WEEKDAY_APPR_PROCESS_START': 'category','HOUR_APPR_PROCESS_START': 'category','REG_REGION_NOT_LIVE_REGION': 'category'
  ,'REG_REGION_NOT_WORK_REGION': 'category','LIVE_REGION_NOT_WORK_REGION': 'category','REG_CITY_NOT_LIVE_CITY': 'category'
  ,'REG_CITY_NOT_WORK_CITY': 'category','LIVE_CITY_NOT_WORK_CITY': 'category','ORGANIZATION_TYPE': 'category'
  ,'FONDKAPREMONT_MODE': 'category','HOUSETYPE_MODE': 'category','WALLSMATERIAL_MODE': 'category', 'EMERGENCYSTATE_MODE': 'category'
  ,'FLAG_DOCUMENT_2': 'category','FLAG_DOCUMENT_3': 'category','FLAG_DOCUMENT_4': 'category', 'FLAG_DOCUMENT_5': 'category'
  ,'FLAG_DOCUMENT_6': 'category','FLAG_DOCUMENT_7': 'category','FLAG_DOCUMENT_8': 'category', 'FLAG_DOCUMENT_9': 'category'
  ,'FLAG_DOCUMENT_10': 'category','FLAG_DOCUMENT_11': 'category','FLAG_DOCUMENT_12': 'category', 'FLAG_DOCUMENT_13': 'category'
  ,'FLAG_DOCUMENT_14': 'category','FLAG_DOCUMENT_15': 'category','FLAG_DOCUMENT_16': 'category', 'FLAG_DOCUMENT_17': 'category'
  ,'FLAG_DOCUMENT_18': 'category','FLAG_DOCUMENT_19': 'category','FLAG_DOCUMENT_20': 'category', 'FLAG_DOCUMENT_21': 'category'
 }

app_data = app_data.astype(dtypes)

0


In [13]:
app_data.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Data columns (total 122 columns):
 #    Column                        Non-Null Count   Dtype   
---   ------                        --------------   -----   
 0    SK_ID_CURR                    307511 non-null  category
 1    TARGET                        307511 non-null  category
 2    NAME_CONTRACT_TYPE            307511 non-null  category
 3    CODE_GENDER                   307511 non-null  category
 4    FLAG_OWN_CAR                  307511 non-null  category
 5    FLAG_OWN_REALTY               307511 non-null  category
 6    CNT_CHILDREN                  307511 non-null  category
 7    AMT_INCOME_TOTAL              307511 non-null  float64 
 8    AMT_CREDIT                    307511 non-null  float64 
 9    AMT_ANNUITY                   307499 non-null  float64 
 10   AMT_GOODS_PRICE               307233 non-null  float64 
 11   NAME_TYPE_SUITE               306219 non-null  category
 12   NAME_INCOME_TY

In [6]:
#Fix missing Values
missed_values = app_data.isna().sum(axis=0) / 307511
print(missed_values.sort_values(ascending=False).head(50)) #no dups for all fields

#40+ Fields have more than half data as null
#We can remove them based on domain knowledge to correct the data
#We can add missing values, many ways, 1- replace mean

COMMONAREA_MEDI                0.70
COMMONAREA_AVG                 0.70
COMMONAREA_MODE                0.70
NONLIVINGAPARTMENTS_MODE       0.69
NONLIVINGAPARTMENTS_AVG        0.69
NONLIVINGAPARTMENTS_MEDI       0.69
FONDKAPREMONT_MODE             0.68
LIVINGAPARTMENTS_MODE          0.68
LIVINGAPARTMENTS_AVG           0.68
LIVINGAPARTMENTS_MEDI          0.68
FLOORSMIN_AVG                  0.68
FLOORSMIN_MODE                 0.68
FLOORSMIN_MEDI                 0.68
YEARS_BUILD_MEDI               0.66
YEARS_BUILD_MODE               0.66
YEARS_BUILD_AVG                0.66
OWN_CAR_AGE                    0.66
LANDAREA_MEDI                  0.59
LANDAREA_MODE                  0.59
LANDAREA_AVG                   0.59
BASEMENTAREA_MEDI              0.59
BASEMENTAREA_AVG               0.59
BASEMENTAREA_MODE              0.59
EXT_SOURCE_1                   0.56
NONLIVINGAREA_MODE             0.55
NONLIVINGAREA_AVG              0.55
NONLIVINGAREA_MEDI             0.55
ELEVATORS_MEDI              

In [ ]:
#EDA
# correlation with Target - target value
corr_features = pd.DataFrame()
corr_features['FI'] = app_data.corr()[label_col]
corr_features['FI_MOD']  = corr_features['FI'].abs()
print(corr_features.sort_values(by='FI_MOD',ascending=True).head(50)) 
#corr_features.style.background_gradient(cmap='viridis')

#We can remove least 10 features whose correltion is less than 1 percent

<ipython-input-38-b7e1186371d7>:4: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr_features['FI'] = app_data.corr()[label_col]


                                FI  FI_MOD
AMT_REQ_CREDIT_BUREAU_WEEK    0.00    0.00
AMT_REQ_CREDIT_BUREAU_HOUR    0.00    0.00
NONLIVINGAPARTMENTS_MODE     -0.00    0.00
AMT_REQ_CREDIT_BUREAU_QRT    -0.00    0.00
SK_ID_CURR                   -0.00    0.00
AMT_REQ_CREDIT_BUREAU_DAY     0.00    0.00
NONLIVINGAPARTMENTS_MEDI     -0.00    0.00
NONLIVINGAPARTMENTS_AVG      -0.00    0.00
AMT_INCOME_TOTAL             -0.00    0.00
OBS_60_CNT_SOCIAL_CIRCLE      0.01    0.01
YEARS_BEGINEXPLUATATION_MODE -0.01    0.01
OBS_30_CNT_SOCIAL_CIRCLE      0.01    0.01
YEARS_BEGINEXPLUATATION_AVG  -0.01    0.01
YEARS_BEGINEXPLUATATION_MEDI -0.01    0.01
LANDAREA_MODE                -0.01    0.01
LANDAREA_AVG                 -0.01    0.01
LANDAREA_MEDI                -0.01    0.01
AMT_REQ_CREDIT_BUREAU_MON    -0.01    0.01
NONLIVINGAREA_MODE           -0.01    0.01
AMT_ANNUITY                  -0.01    0.01
NONLIVINGAREA_MEDI           -0.01    0.01
NONLIVINGAREA_AVG            -0.01    0.01
COMMONAREA_

In [ ]:
# outlier check
plt.figure(figsize=(15, 20))

for i, c in enumerate(app_data.drop(['SK_ID_CURR', 'TARGET'], axis=1).select_dtypes(exclude='category').columns):
    plt.subplot(10,2,i*2+1)
    sns.boxplot(app_data[c], color='blue')
    plt.title('Distribution plot for field:' + c)
    plt.xlabel('')
    plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)
    
    # plt.subplot(10,2,i*2+2)
    # sns.boxplot(app_data[c].apply('log1p'), color='red')
    # plt.title('Log1p distribution plot for field:' + c)
    # plt.xlabel('')
    # plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)

In [ ]:
plt.figure(figsize=(15, 12))

for i, c in enumerate(app_data.select_dtypes(exclude='category').columns):
    plt.subplot(5,2,i+1)
    sns.kdeplot(app_data[c])
    plt.title('Distribution plot for field:' + c)
    plt.xlabel('')
    plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)

# Encoding Labels

In [ ]:
# from sklearn.preprocessing import LabelEncoder, RobustScaler
# le = LabelEncoder()
# app_data['gender'] = le.fit_transform(app_data['gender'])
# app_data.gender.head()

In [ ]:
# rs = RobustScaler()
# for c in df[['age', 'gender', 'total_bilirubin', 'alkaline_phosphotase', 'alamine_aminotransferase', 'albumin_and_globulin_ratio']].columns:
#     df[c] = rs.fit_transform(df[c].values.reshape(-1, 1))
# df.head()

In [ ]:
app_data.describe(include='all').T

(307511, 122)

Split Data and Model

In [25]:
app_data[label_col].value_counts()

0    282686
1     24825
Name: TARGET, dtype: int64

In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(app_data[feature_columns], app_data[label_col], test_size=0.2, random_state=123)

print('Train values shape:', X_train.shape)
print('Test values shape:', X_test.shape)
print('Train target shape:', y_train.shape)
print('Test target shape:', y_test.shape)

Train values shape: (246008, 120)
Test values shape: (61503, 120)
Train target shape: (246008,)
Test target shape: (61503,)


In [26]:
from sklearn.utils import class_weight
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

model = XGBClassifier(random_state=123,tree_method="hist",enable_categorical='True',class_weight={0:0.92,1:0.08})
model.fit(X_train, y_train)
y_train_hat = model.predict(X_train)
y_test_hat = model.predict(X_test)

print(model)
print('Train performance')
print('-------------------------------------------------------')
print(classification_report(y_train, y_train_hat))

print('Test performance')
print('-------------------------------------------------------')
print(classification_report(y_test, y_test_hat))

print('Roc_auc score')
print('-------------------------------------------------------')
print(roc_auc_score(y_test, y_test_hat))
print('')

print('Confusion matrix')
print('-------------------------------------------------------')
print(confusion_matrix(y_test, y_test_hat))

[18:33:41] WARNING: ../src/learner.cc:767: 
Parameters: { "class_weight" } are not used.

XGBClassifier(base_score=None, booster=None, callbacks=None,
              class_weight={0: 0.92, 1: 0.08}, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None,
              early_stopping_rounds=None, enable_categorical='True',
              eval_metric=None, feature_types=None, gamma=None, gpu_id=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, ...)
Train performance
-------------------------------------------------------
              precision    recall  f1-score   support

